In [3]:
import pandas as pd 
import pysam
import json

In [ ]:
df = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230615AMU/BCRLFBVC220616037X1CK/03.assemble/BCRLFBVC220616037X1CK/outs/filtered_contig_annotations.csv")

In [ ]:
df = df[df["productive"]==True]
df = df[df["chain"]!="Multi"]

In [ ]:
df = df.sort_values(["barcode", "chain"])

In [ ]:
#df

In [ ]:
grouped = df.groupby('barcode')

In [ ]:
filtered_contig_id = []
for _, group_data in grouped:
    if len(group_data)==2 and len(group_data["chain"].unique()) == 2:
        if "IGH" in group_data["chain"].unique():
            #print(group_data["chain"].unique())
            filtered_contig_id.extend(list(group_data["contig_id"]))

In [ ]:
filtered_contig_id

In [ ]:
df = df[df["contig_id"].isin(filtered_contig_id)]
df["chain_type"] = df[["chain","c_gene"]].apply(':'.join, axis=1)
chain_dict = df.set_index("contig_id").to_dict(orient="dict")["chain_type"]

In [ ]:
#chain_dict

In [ ]:
contig_dict = {}
with pysam.FastxFile("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230615AMU/BCRLFBVC220616037X1CK/03.assemble/BCRLFBVC220616037X1CK/outs/filtered_contig.fasta") as f:
    for read in f :
        contig_dict[read.name] = read.sequence

In [ ]:
#contig_dict

In [ ]:
with open("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230615AMU/BCRLFBVC220616037X1CK/02.convert/barcode_convert.json", 'r') as f:
    tenX_sgr = json.load(f)

In [ ]:
#tenX_sgr

In [ ]:
pair_fasta = open("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230615AMU/paired_contig.fasta", 'w')
for contig in filtered_contig_id:
    pair_fasta.write(f">{contig} | {chain_dict[contig]}\n")
    pair_fasta.write(f"{contig_dict[contig]}\n")
pair_fasta.close()

In [ ]:
# IGHD change C to IGHG1

In [6]:
df = pd.read_csv("/SGRNJ06/bioinfo/PROJ04/PROJ_22.Other/P230331001_fl-BCR/T_117-B_c3/T_117-B/03.assemble/T_117-B/outs/filtered_contig_annotations.csv")
df = df[df["productive"]==True]
df = df[df["chain"]!="Multi"]
df = df.sort_values(["barcode", "chain"])
grouped = df.groupby('barcode')
filtered_contig_id = []
for _, group_data in grouped:
    if len(group_data)==2 and len(group_data["chain"].unique()) == 2:
        if "IGH" in group_data["chain"].unique():
            #print(group_data["chain"].unique())
            filtered_contig_id.extend(list(group_data["contig_id"]))
with open("/SGRNJ06/bioinfo/PROJ04/PROJ_22.Other/P230331001_fl-BCR/T_117-B_c3/T_117-B/02.convert/barcode_convert.json", 'r') as f:
    tenX_sgr = json.load(f)

In [169]:
airr = pd.read_csv("/SGRNJ06/bioinfo/PROJ04/PROJ_22.Other/P230331001_fl-BCR/T_117-B_c3/T_117-B/03.assemble/T_117-B/outs/airr_rearrangement.tsv", sep='\t')
IGHC_c = airr[airr["c_call"]=="IGHG1"]
start = int(IGHC_c["c_sequence_start"].unique()[0])
end = int(IGHC_c["c_sequence_end"].unique()[0])
IGHC_c_seq = IGHC_c["sequence"].unique()[0][start-1: end]
IGHC_c_seq

'CCAAAACGACACCCCCATCTGTCTATCCACTG'

In [170]:
airr = airr[airr["sequence_id"].isin(filtered_contig_id)]
airr['cell_id'] = airr['cell_id'].apply(lambda x: tenX_sgr[x.split('-')[0]])
airr['sequence_id'] = airr['sequence_id'].apply(lambda x: tenX_sgr[x.split('-')[0]]+'_'+x.split('_')[1]+'_'+x.split('_')[2])

In [171]:
airr

,cell_id,clone_id,sequence_id,sequence,sequence_aa,productive,rev_comp,v_call,v_cigar,d_call,...,v_sequence_end,d_sequence_start,d_sequence_end,j_sequence_start,j_sequence_end,c_sequence_start,c_sequence_end,consensus_count,duplicate_count,is_cell
0,CTGAGCCATGGCTTCATGGCTTCA,NaN,CTGAGCCATGGCTTCATGGCTTCA_contig_1,GACCACCGAGATCTACACTCGTCGGCAGCGTCAGATGTGTATAAGA...,MEWTWVFLFLLSVTAGVHSQVQLQQSGAELMKPGASVKISCKATGY...,T,F,IGHV1-9,242S351M87S,NaN,...,593,NaN,NaN,610,657,658.0,680.0,42057,404,T
1,CTGAGCCATGGCTTCATGGCTTCA,NaN,CTGAGCCATGGCTTCATGGCTTCA_contig_2,GACCACCGAGATCTACACTCGTCGGCAGCGTCAGATGTGTATAAGA...,MDFQVQIFSFLLISASVIISRGQIVLTQSPAIMSASPGEKVTMTCS...,T,F,IGKV4-59,238S352M106S,NaN,...,590,NaN,NaN,585,623,624.0,696.0,32254,482,T
2,CGGATTGCAGCAGGAACAAGGAGC,NaN,CGGATTGCAGCAGGAACAAGGAGC_contig_1,GGGGACCAATATTGAAAAGAATAGACCTGGTTTGTGAATTATGGCC...,MAWISLILSLLALSSGAISQAVVTQESALTTSPGETVTLTCRSSTG...,T,F,IGLV1,40S351M83S,NaN,...,391,NaN,NaN,388,425,426.0,474.0,11125,111,T
3,CGGATTGCAGCAGGAACAAGGAGC,NaN,CGGATTGCAGCAGGAACAAGGAGC_contig_2,GCTCCTTGGACTAGGAGTCAGCATTTCCTGCTCGCGTTAGTGACAA...,MKLWLNWIFLVTLLNGIQCEVKLVESGGGLVQPGGSLRLSCATSGF...,T,F,IGHV7-3,208S361M74S,NaN,...,569,NaN,NaN,583,620,621.0,643.0,4535,31,T
8,CGACACACAAGAGATCAGCACCTC,NaN,CGACACACAAGAGATCAGCACCTC_contig_1,AGACATAGGAGCTTGGCTCTGGTTCCCAAGACCTCTCACTCACTTC...,MEWELSLIFIFALLKDVQCEVQLLETGGGLVQPGGSRGLSCEGSGF...,T,F,IGHV11-2,51S353M82S,NaN,...,404,NaN,NaN,411,463,464.0,486.0,5136,35,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23179,AAGACGGAGACTAGTACTGAGCCA,NaN,AAGACGGAGACTAGTACTGAGCCA_contig_2,ACACAGACCACTCACCATGGACTCCAGGCTCAATTTAGTTTTCCTT...,MDSRLNLVFLVLILKGVQCDVQLVESGGGLVQPGGSRKLSCAASGF...,T,F,IGHV5-17,16S351M93S,NaN,...,367,NaN,NaN,390,437,438.0,460.0,13,2,T
23186,CAAGGAGCCCATCCTCCACCTTAC,NaN,CAAGGAGCCCATCCTCCACCTTAC_contig_1,CTGGAATTGATTCCCAGTTCCTCACGTTCAGTGATGAGCAGTGAAC...,MSSEHRPLTMNFGLRLIFLVLTLKGVQCDVKLVESGGGLVKPGGSL...,T,F,IGHV5-9-1,33S380M88S,NaN,...,413,NaN,NaN,431,478,479.0,501.0,24,2,T
23187,CAAGGAGCCCATCCTCCACCTTAC,NaN,CAAGGAGCCCATCCTCCACCTTAC_contig_2,ATGGCCTGGATTTCACTTATACTCTCTCTCCTGGCTCTCAGCTCAG...,MAWISLILSLLALSSGAISQAVVTQESALTTSPGETVTLTCRSSTG...,T,F,IGLV1,351M87S,NaN,...,351,NaN,NaN,348,385,386.0,438.0,15,4,T
23199,AGCACCTCTCTTCACATCTTCACA,NaN,AGCACCTCTCTTCACATCTTCACA_contig_1,TAATAGACTTGGTTTGTGAATTATGGCCTGGACTTCACTTATACTC...,MAWTSLILSLLALCSGASSQAVVTQESALTTSPGGTVILTCRSSTG...,T,F,IGLV2,22S351M87S,NaN,...,373,NaN,NaN,370,407,408.0,460.0,17,4,T


In [172]:
IGHD_contig = airr[airr["c_call"]=="IGHD"]
other_contig = airr[airr["c_call"]!="IGHD"]

In [173]:
IGHD_contig["c_call"] = "IGHG1"
IGHD_contig["c_call"][25]

/SGRNJ/Public/Software/conda_env/chenjunjie/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'IGHG1'

In [174]:
# 恒定区
IGHD_contig["c_region_seq"] = "CCAAAACGACACCCCCATCTGTCTATCCACTG"

c_start = list(other_contig["j_sequence_end"])
seq_nt = list(other_contig["sequence"])
c_seq = []
for i in range(len(c_start)):
    newseq = seq_nt[i][int(c_start[i]):]
    c_seq.append(newseq)

other_contig["c_region_seq"] = c_seq

# 可变区
res_df = pd.concat([IGHD_contig, other_contig])
v_region_start = list(res_df["v_sequence_start"])
v_region_end = list(res_df["j_sequence_end"])
seq_nt = list(res_df["sequence"])
vdj_seq = []

for i in range(len(v_region_start)):
    newseq = seq_nt[i][int(v_region_start[i])-1 : int(v_region_end[i])]
    vdj_seq.append(newseq)

res_df["v_region_seq"] = vdj_seq

/SGRNJ/Public/Software/conda_env/chenjunjie/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/SGRNJ/Public/Software/conda_env/chenjunjie/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [175]:
res_df["chain"] = res_df["v_call"].apply(lambda x: str(x[:3]))
res_df["c_call"] = res_df["c_call"].astype(str)
res_df = res_df[["cell_id", "sequence_id", "chain", "c_call", "v_region_seq", "c_region_seq"]]
res_df = res_df.sort_values(["cell_id", "chain"])

In [176]:
res_df

,cell_id,sequence_id,chain,c_call,v_region_seq,c_region_seq
5663,AAACATCGAACAACCAAACAACCA,AAACATCGAACAACCAAACAACCA_contig_1,IGH,IGHG1,ATGGATTGGCTGTGGAACTTGCTATTCCTGATGGCAGCTGCCCAAA...,CCAAAACGACACCCCCATCTGTCTATCCACTG
5664,AAACATCGAACAACCAAACAACCA,AAACATCGAACAACCAAACAACCA_contig_2,IGK,IGKC,ATGGTATCCACACCTCAGTTCCTTGTATTTTTGCTTTTCTGGATTC...,GGGCTGATGCTGCACCAACTGTATCCATCTTCCCACCATCCAGTGA...
8168,AAACATCGAACAACCAGCCAAGAC,AAACATCGAACAACCAGCCAAGAC_contig_1,IGH,IGHM,ATGGAATGGAGCGGGGTCTTTATCTTTCTCCTGTCAGTAACTGCAG...,AGAGTCAGTCCTTCCCAAATGTC
8169,AAACATCGAACAACCAGCCAAGAC,AAACATCGAACAACCAGCCAAGAC_contig_2,IGK,IGKC,ATGGAATCACAGACCCAGGTCCTCATGTTTCTTCTGCTCTGGGTAT...,GGGCTGATGCTGCACCAACTGTATCCATCTTCCCACCATCCAGTGA...
22028,AAACATCGAACCGAGACATACCAA,AAACATCGAACCGAGACATACCAA_contig_1,IGH,nan,ATGGATTGGCTGTGGAACTTGCTATTCCTGATGGCAGCTGCCCAAA...,GTGATAATAAG
...,...,...,...,...,...,...
12162,TTCACGCATGGTGGTAACGTATCA,TTCACGCATGGTGGTAACGTATCA_contig_1,IGK,IGKC,ATGACCATGTTCTCACTAGCTCTTCTCCTCAGTCTTCTTCTCCTCT...,GGGCTGATGCTGCACCAACTGTATCCATCTTCCCACCATCCAGTGA...
22358,TTCACGCATTCACGCAAACTCACC,TTCACGCATTCACGCAAACTCACC_contig_1,IGH,IGHG1,ATGGAATGGAGTTGGATATTTCTCTTTCTCCTGTCAGGAACTGCAG...,CCAAAACGACACCCCCATCTGTCTATCCACTG
22359,TTCACGCATTCACGCAAACTCACC,TTCACGCATTCACGCAAACTCACC_contig_2,IGK,IGKC,ATGGAGTCACAGACTCAGGTCTTTGTATACATGTTGCTGTGGTTGT...,GGGCTGATGCTGCACCAACTGTATCCATCTTCCCACCATCCAGTGA...
431,TTCACGCATTCACGCAGTCGTAGA,TTCACGCATTCACGCAGTCGTAGA_contig_2,IGH,IGHM,ATGGAATGGAGCTGGATCTTTCTCTTCCTCCTGTCAGGAACTACAG...,AGAGTCAGTCCTTCCCAAATGTC


In [177]:
res_df.to_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230627AMU/paired_contig.tsv", sep='\t', index=False)

In [ ]:
IGHD_contig_id, IGHD_seq, IGHD_contig_start, IGHD_contig_end = list(IGHD_contig["sequence_id"]), list(IGHD_contig["sequence"]), list(IGHD_contig["c_sequence_start"]), list(IGHD_contig["c_sequence_end"])

In [ ]:
changed_IGHD_seq = []
for i in range(len(IGHD_contig_id)):
    newseq = IGHD_seq[i][:int(IGHD_contig_start[i])-1] + IGHC_c_seq
    changed_IGHD_seq.append(newseq)

In [ ]:
IGHD_seq[0]

In [ ]:
len(IGHD_seq[0])

In [ ]:
changed_IGHD_seq[0]

In [ ]:
len(changed_IGHD_seq[0])

In [ ]:
IGHD_contig["sequence"] = changed_IGHD_seq

In [ ]:
res_df = pd.concat([IGHD_contig, other_contig])

In [ ]:
res_df

In [ ]:
res_df["chain"] = res_df["v_call"].apply(lambda x: str(x[:3]))

In [ ]:
res_df["c_call"] = res_df["c_call"].astype(str)

In [ ]:
res_df["chain_type"] = res_df[["chain","c_call"]].apply(':'.join, axis=1)

In [ ]:
# res_df.to_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230615AMU/test.tsv", sep='\t', index=False)

In [ ]:
chain_dict = res_df.set_index("sequence_id").to_dict(orient="dict")["chain_type"]
seq_dict = res_df.set_index("sequence_id").to_dict(orient="dict")["sequence"]

In [ ]:
chain_dict

In [ ]:
res_df = res_df.sort_values(["cell_id", "chain"])

In [ ]:
contig_dict

In [ ]:
filtered_contig_id = list(res_df.sequence_id)

In [ ]:
pair_fasta = open("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230615AMU/paired_contig.fasta", 'w')
for contig in filtered_contig_id:
    pair_fasta.write(f">{contig} | {chain_dict[contig]}\n")
    pair_fasta.write(f"{seq_dict[contig]}\n")
pair_fasta.close()

In [ ]:
pair_fasta.close()